In [7]:
import sys
import os

# Add the path to the directory where your module is located
module_path = os.path.abspath(os.path.join('..', 'C:\\Users\\pier1\\OneDrive\\Desktop\\uni\\Master\\2.Semester\\Machine Learning (WIWI)\\Project\\Data for depression\\'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module
from helpers import *

In [8]:
import pandas as pd
import numpy as np
from itertools import combinations
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from NN import *

In [46]:
df = pd.read_csv('..\data\Threshold_3_Operator_-_Depressionfeature_BP_PHQ_9_PercentofDataset_100.csv')
print_information(df)
# 0.9 train, 0.1 test
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
# 0.8 train, 0.2 validation
train_df, validation_df = train_test_split(train_df, train_size=0.8, random_state=42)

train_features, train_targets = df_to_tensor(train_df,features_column='FEATURES', target_col='Depression')
validation_features, validation_targets = df_to_tensor(validation_df, features_column='FEATURES', target_col='Depression')
test_features, test_targets = df_to_tensor(test_df, features_column='FEATURES', target_col='Depression')

print(train_features.shape, train_targets.shape, validation_features.shape, validation_targets.shape, test_features.shape, test_targets.shape)

train_dataset = CustomDataset(train_features, train_targets)
validation_dataset = CustomDataset(validation_features, validation_targets)
test_dataset = CustomDataset(test_features, test_targets)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

-9.0
ID_1          203
ID_2          203
group_id      203
SEX           203
AGE           203
             ... 
FEATURE_51    203
FEATURE_52    203
FEATURE_53    203
FEATURE_54    203
FEATURE_55    203
Length: 65, dtype: int64
-8.0
ID_1          149
ID_2          149
group_id      149
SEX           149
AGE           149
             ... 
FEATURE_51    149
FEATURE_52    149
FEATURE_53    149
FEATURE_54    149
FEATURE_55    149
Length: 65, dtype: int64
-6.0
ID_1          146
ID_2          146
group_id      146
SEX           146
AGE           146
             ... 
FEATURE_51    146
FEATURE_52    146
FEATURE_53    146
FEATURE_54    146
FEATURE_55    146
Length: 65, dtype: int64
-3.0
ID_1          403
ID_2          403
group_id      403
SEX           403
AGE           403
             ... 
FEATURE_51    403
FEATURE_52    403
FEATURE_53    403
FEATURE_54    403
FEATURE_55    403
Length: 65, dtype: int64
-2.0
ID_1          460
ID_2          460
group_id      460
SEX           460
AGE        

In [47]:
# Configuration
input_size = 56
hidden_size = 128
num_epochs = 30
learning_rate_values = [0.001,0.003,0.006,0.01]
momentum_values = [0.9, 0.95,0.99]
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Model is bad at 94 % accuracy, because 94 percent of the data is not depressed (PHQ_9)
# Initialize the model, criterion, and optimizer
#model = Depression_Classifier_v_0(input_size, hidden_size).to(device)
results = {}
#criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid and binary cross-entropy loss
criterion = nn.BCELoss()
optimizers = ['Adam', 'SGD']

for i in range(3):
    for optimizer in optimizers:
        if optimizer == 'Adam':
            for learning_rate in learning_rate_values:
                if i == 0:
                    m = Depression_Classifier_v_0(input_size, hidden_size).to(device)
                elif i == 1:
                    m = Depression_Classifier_v_1(input_size, hidden_size).to(device)
                else:
                    m = Depression_Classifier_v_2(input_size, hidden_size).to(device)

                o = optim.Adam(m.parameters(), lr=learning_rate)

                train_model_cross_validation(m, train_dataloader,validation_dataloader,criterion, o, num_epochs, device)

                validation_loss, accuracy = evaluate_model(m, test_dataloader, criterion, device)

                # Store results
                results[(f'Model_{i}', optimizer, learning_rate)] = {
                    'validation_loss': validation_loss,
                    'accuracy': accuracy
                }
        else:
            for momentum, learning_rate in itertools.product(momentum_values, learning_rate_values):
                if i == 0:
                    m = Depression_Classifier_v_0(input_size, hidden_size).to(device)
                elif i == 1:
                    m = Depression_Classifier_v_1(input_size, hidden_size).to(device)
                else:
                    m = Depression_Classifier_v_2(input_size, hidden_size).to(device)
                    
                o = optim.SGD(m.parameters(), lr=learning_rate, momentum=momentum)

                train_model_cross_validation(m, train_dataloader,validation_dataloader,criterion, o, num_epochs, device)

                validation_loss, accuracy = evaluate_model(m, test_dataloader, criterion, device)

                # Store results
                results[(f'Model_{i}', optimizer, learning_rate,momentum)] = {
                    'validation_loss': validation_loss,
                    'accuracy': accuracy
                }
                

Epoch [1/30], Training Loss: 0.3588, Validation Loss: 0.3739
Epoch [2/30], Training Loss: 0.3335, Validation Loss: 0.3770
Epoch [3/30], Training Loss: 0.3228, Validation Loss: 0.3533
Epoch [4/30], Training Loss: 0.3134, Validation Loss: 0.3531
Epoch [5/30], Training Loss: 0.3082, Validation Loss: 0.3504
Epoch [6/30], Training Loss: 0.3011, Validation Loss: 0.3304
Epoch [7/30], Training Loss: 0.2955, Validation Loss: 0.3342
Epoch [8/30], Training Loss: 0.2891, Validation Loss: 0.3237
Epoch [9/30], Training Loss: 0.2844, Validation Loss: 0.3203
Epoch [10/30], Training Loss: 0.2796, Validation Loss: 0.3125
Epoch [11/30], Training Loss: 0.2745, Validation Loss: 0.3252
Epoch [12/30], Training Loss: 0.2720, Validation Loss: 0.3060
Epoch [13/30], Training Loss: 0.2697, Validation Loss: 0.3059
Epoch [14/30], Training Loss: 0.2645, Validation Loss: 0.3177
Epoch [15/30], Training Loss: 0.2649, Validation Loss: 0.3022
Epoch [16/30], Training Loss: 0.2602, Validation Loss: 0.3010
Epoch [17/30], Tr

KeyboardInterrupt: 

In [32]:
print(results)

{('Model_0', 'Adam', 0.001): {'validation_loss': 0.42725964880629674, 'accuracy': 82.09143457698372}, ('Model_0', 'Adam', 0.002): {'validation_loss': 0.4523360274682109, 'accuracy': 80.45191802417236}, ('Model_0', 'Adam', 0.005): {'validation_loss': 0.48782783526702217, 'accuracy': 78.66526537046768}, ('Model_0', 'Adam', 0.01): {'validation_loss': 0.5232279593712531, 'accuracy': 76.04834471886495}, ('Model_0', 'SGD', 0.001, 0.9): {'validation_loss': 0.4867494810347589, 'accuracy': 78.5496584340515}, ('Model_0', 'SGD', 0.002, 0.9): {'validation_loss': 0.4868668759829246, 'accuracy': 78.2764056752496}, ('Model_0', 'SGD', 0.005, 0.9): {'validation_loss': 0.451741439384102, 'accuracy': 80.69364161849711}, ('Model_0', 'SGD', 0.01, 0.9): {'validation_loss': 0.44255475809910155, 'accuracy': 80.90383604834471}, ('Model_0', 'SGD', 0.001, 0.95): {'validation_loss': 0.5041491652795133, 'accuracy': 78.2238570677877}, ('Model_0', 'SGD', 0.002, 0.95): {'validation_loss': 0.45157863334721365, 'accura

In [37]:
df = pd.DataFrame()
models_df = []
optimizer_df = []
learning_rate_df = []
momentum_df = []
accuracy_df = []
validation_loss_df = []
for key,value in results.items():
    accuracy_df.append(value['accuracy'])
    validation_loss_df.append(value['validation_loss'])
    opti = key[1]
    models_df.append(key[0])
    optimizer_df.append(opti)
    learning_rate_df.append(key[2])
    if opti == 'SGD':
        momentum_df.append(key[3])
    else:
        momentum_df.append(None)
        
df['Model'] = models_df
df['Optimizer'] = optimizer_df
df['Learning Rate'] = learning_rate_df
df['Momentum'] = momentum_df
df['Validation Loss'] = validation_loss_df
df['Accuracy'] = accuracy_df


In [38]:
print(df)

      Model Optimizer  Learning Rate  Momentum  Validation Loss   Accuracy
0   Model_0      Adam          0.001       NaN         0.427260  82.091435
1   Model_0      Adam          0.002       NaN         0.452336  80.451918
2   Model_0      Adam          0.005       NaN         0.487828  78.665265
3   Model_0      Adam          0.010       NaN         0.523228  76.048345
4   Model_0       SGD          0.001      0.90         0.486749  78.549658
5   Model_0       SGD          0.002      0.90         0.486867  78.276406
6   Model_0       SGD          0.005      0.90         0.451741  80.693642
7   Model_0       SGD          0.010      0.90         0.442555  80.903836
8   Model_0       SGD          0.001      0.95         0.504149  78.223857
9   Model_0       SGD          0.002      0.95         0.451579  80.304782
10  Model_0       SGD          0.005      0.95         0.454234  80.430899
11  Model_0       SGD          0.010      0.95         0.461556  79.348397
12  Model_0       SGD    

In [39]:
df.to_csv('FNN_BP_PHQ_9_minus.csv', index=False)

Use the csv data to compare the different models and cross-validations

Use the models to predict the test data and use the confusion matrix and their metrices (accurary, precision, recall and f1-score)